In [17]:
import cv2
import mediapipe as mp
import pandas as pd
import os
import math

In [19]:
import cv2
import math
import pandas as pd
import os
import mediapipe as mp

# Variables globales para coordenadas y flags
x1, y1, x2, y2 = 0, 0, 0, 0
flag_click1, flag_click2 = False, False

def distancia_landmark(x1, y1, x2, y2):
    
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def click_distancia1(event, x, y, flags, params):
    """
    Callback para el primer punto de clic en el video.

    Args:
        event: Tipo de evento de mouse.
        x: Coordenada X del clic.
        y: Coordenada Y del clic.
        flags: Flags del evento de mouse.
        params: Parámetros adicionales.
    """
    global x1, y1, flag_click1
    if event == cv2.EVENT_LBUTTONDOWN:
        x1, y1 = x, y
        flag_click1 = True

def click_distancia2(event, x, y, flags, params):
    """
    Callback para el segundo punto de clic en el video.

    Args:
        event: Tipo de evento de mouse.
        x: Coordenada X del clic.
        y: Coordenada Y del clic.
        flags: Flags del evento de mouse.
        params: Parámetros adicionales.
    """
    global x2, y2, flag_click2
    if event == cv2.EVENT_LBUTTONDOWN:
        x2, y2 = x, y
        flag_click2 = True

def click_event(event, x, y, flags, params):
    """
    Callback para eventos de clic en el video.

    Args:
        event: Tipo de evento de mouse.
        x: Coordenada X del clic.
        y: Coordenada Y del clic.
        flags: Flags del evento de mouse.
        params: Parámetros adicionales.
    """
    if event == cv2.EVENT_LBUTTONDOWN:
        print('Posición en X:', x, 'Posición en Y:', y)
    if event == cv2.EVENT_RBUTTONDOWN:
        print(x, ' ', y)

# Inicializar MediaPipe Pose y las utilidades de dibujo
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Crear una lista vacía para almacenar los datos del landmark
landmark_data = []

# Abrir el archivo de video
cap = cv2.VideoCapture("vid/videotracker.mp4")

# Variable de pausa
is_paused = False
last_frame = None

# Inicializar la detección de pose
with mp_pose.Pose(
        static_image_mode=False,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
) as pose:
    frame_count = 0
    while True:
        if not is_paused:
            ret, frame = cap.read()
            if not ret:
                break
            last_frame = frame
        else:
            frame = last_frame

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frame_rgb)

        if results.pose_landmarks:
            landmark_indices = [26, 28]  # Lista de IDs de landmarks que deseas mostrar
            h, w, _ = frame.shape
            for landmark_idx in landmark_indices:
                if landmark_idx < len(results.pose_landmarks.landmark):
                    landmark = results.pose_landmarks.landmark[landmark_idx]
                    cx, cy = int(landmark.x * w), int(landmark.y * h)
                    cv2.circle(frame, (cx, cy), 5, (0, 20, 255), cv2.FILLED)
                    
                    landmark_data.append({
                        'frame': frame_count,
                        'landmark_id': landmark_idx,
                        'x': landmark.x,
                        'y': landmark.y,
                        'z': landmark.z,
                        'visibility': landmark.visibility
                    })

        cv2.imshow("frame", frame)
        cv2.setMouseCallback('frame', click_event)
        key = cv2.waitKey(1) & 0xFF
        ###### LO NUEVO ######
        #Definimos una tecla nueva para medir la distancia entre dos clicks en pixeles ( falta pasar a cm, como todo)
        #para poder utilizarla mas adelante con la finalidad de pasar a centimetros de manera generica para todos los videos 
        if key == ord('d'):
            flag_click1 = False
            flag_click2 = False
            print('Click en primer punto')
            while not flag_click1:
                cv2.setMouseCallback('frame', click_distancia1)
                cv2.waitKey(1)  # Necesario para permitir el procesamiento de eventos
            print('Click en segundo punto')
            while not flag_click2:
                cv2.setMouseCallback('frame', click_distancia2)
                cv2.waitKey(1)  # Necesario para permitir el procesamiento de eventos
            distancia = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
            print(f'La distancia es de: {distancia} (en píxeles)')

        if key == ord('p'):
            is_paused = not is_paused
        elif key == 27:
            break

        if not is_paused:
            frame_count += 1
        #### FIN LO NUEVO #####
cap.release()
cv2.destroyAllWindows()

df = pd.DataFrame(landmark_data)
output_folder = "output"
os.makedirs(output_folder, exist_ok=True)
csv_file_path = os.path.join(output_folder, 'tracked_landmarks.csv')
df.to_csv(csv_file_path, index=False)

print(f"Datos guardados en: {csv_file_path}")


error: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'arrowedLine'
> Overload resolution failed:
>  - arrowedLine() missing required argument 'pt1' (pos 2)
>  - arrowedLine() missing required argument 'pt1' (pos 2)


In [ ]:
# Cargar el CSV original con todos los landmarks trackeados
csv_file_path = 'output/tracked_landmarks.csv'  # Cambia esta ruta si es necesario
df = pd.read_csv(csv_file_path)
landmark_id = 28;

# Filtrar solo las filas donde landmark_id es 28
df_landmark_27 = df[df['landmark_id'] == landmark_id]

# Crear una carpeta para guardar el nuevo archivo CSV (si no existe ya)
output_folder = "output"
os.makedirs(output_folder, exist_ok=True)

# Guardar el dataframe filtrado en un nuevo archivo CSV llamado 'trackeo_deseado.csv'
csv_file_path_landmark = os.path.join(output_folder, 'trackeo_deseado.csv')
df_landmark_27.to_csv(csv_file_path_landmark, index=False)

print(f"Datos del landmark 28 guardados en: {csv_file_path_landmark}")

Datos del landmark 28 guardados en: output\trackeo_deseado.csv
